In [53]:
import pandas as pd
import nfl_data_py as nfl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from pulp import *
import gurobipy as gp


In [71]:
player_data = pd.read_csv('/Users/mariu/VS Code Repositories/data-driven-fantasy-manager/player_data.csv')
players_rank = pd.read_csv('/Users/mariu/VS Code Repositories/data-driven-fantasy-manager/rank.csv')

In [4]:
# Spieplan Manager
def create_round_robin(players):
    n = len(players)
    schedule = []

    for round in range(n-1):
        for i in range(n//2):
            home = players[i]
            away = players[n - i - 1 - round]
            if i % 2 == round % 2:
                home, away = away, home
            schedule.append((round + 1, home, away))

    return schedule

def add_extra_games(schedule, players):
    n = len(players)
    rounds = len(schedule) // (n // 2)
    
    for round in range(rounds, rounds + (n - 1)//2):
        for i in range(n//2):
            home = players[i]
            away = players[n - i - 1]
            if i % 2 == round % 2:
                home, away = away, home
            schedule.append((round + 1, home, away))
    
    return schedule

managers = ['Manager_' + str(i) for i in range(1, 11)]

round_robin_schedule = create_round_robin(managers)
full_schedule = add_extra_games(round_robin_schedule, managers)

manager_schedule = pd.DataFrame(full_schedule, columns=['week', 'Manager', 'Opponent'])
manager_schedule.sort_values(by=['week', 'Manager'], inplace=True)

In [65]:
# Spieler-ID und Woche als Index festlegen
player_data.set_index(['player_id', 'week'], inplace=True)

# Erstellen einer Matrix der erwarteten Punkte
expected_points = player_data['predicted_fantasy_points'].unstack()

# NaN-Werte ersetzen (falls ein Spieler in einer bestimmten Woche nicht spielt)
expected_points.fillna(0, inplace=True)


player_data = player_data.reset_index()
expected_points_long = expected_points.reset_index().melt(id_vars='player_id', var_name='week', value_name='points')
expected_points_long['week'] = expected_points_long['week'].astype(int)  # Stellen Sie sicher, dass 'week' ein integer ist
expected_points_dict = {(row['player_id'], row['week']): row['points'] for index, row in expected_points_long.iterrows()}


In [34]:
# Angenommen, die Teams sind durchnummeriert (z.B. 'Manager_1', 'Manager_2', ..., 'Manager_10')
teams = ['Manager_' + str(i+1) for i in range(10)]

# Die Anfangsreihenfolge für den Draft kann zufällig oder auf eine andere Weise festgelegt werden
initial_draft_order = teams

# Erstelle die Snake-Draft-Order für alle Runden
draft_order = []
for r in range(16): # Angenommen, wir haben 14 Runden im Draft
    if r % 2 == 0:
        draft_order.append(initial_draft_order)
    else:
        draft_order.append(initial_draft_order[::-1]) # Die Reihenfolge umkehren


In [72]:
players_rank

,Unnamed: 0,Rank,POS,AVG,Sleeper,Player,Team,Bye,player_id,predicted_fantasy_points,position_group,replacement_value,vor,rank
0,10,11,WR4,10.7,NaN,Michael Thomas,NO,9.0,00-0032765,258.939500,WR,17.357178,241.582336,1.0
1,9,10,WR3,10.5,NaN,Julio Jones,NaN,NaN,00-0027944,241.330350,WR,17.357178,223.973175,2.0
2,4,5,WR1,5.3,NaN,DeAndre Hopkins,NaN,NaN,00-0030564,239.952260,WR,17.357178,222.595078,3.0
3,11,12,WR5,12.8,NaN,Odell Beckham,BAL,8.0,00-0031235,237.120970,WR,17.357178,219.763794,4.0
4,81,93,WR39,97.8,NaN,Emmanuel Sanders,NaN,NaN,00-0027685,231.151870,WR,17.357178,213.794693,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,441,672,QB87,541.0,NaN,Sean Mannion,NaN,NaN,00-0032245,19.976408,QB,221.561752,-201.585344,398.0
398,489,783,QB98,662.0,NaN,Mike Glennon,NaN,NaN,00-0030520,17.617748,QB,221.561752,-203.944004,399.0
399,543,1042,QB127,962.0,NaN,Nick Mullens,MIN,12.0,00-0033319,14.515570,QB,221.561752,-207.046183,400.0
400,401,574,QB63,446.0,NaN,AJ McCarron,NaN,NaN,00-0031288,12.610646,QB,221.561752,-208.951106,401.0


In [7]:
from itertools import cycle

# Erstelle einen round-robin Spielplan
schedule = {}
rotating_teams = teams[1:] # 'Manager_1' ist fixiert, die anderen Teams rotieren

for week in range(1, 14): # Angenommen, wir haben eine 14-wöchige Saison
    rotating_teams = rotating_teams[-1:] + rotating_teams[:-1] # rotiere die Teams
    matchups = list(zip(['Manager_1'] + rotating_teams[:len(rotating_teams)//2], rotating_teams[len(rotating_teams)//2:]))
    
    for team1, team2 in matchups:
        if team1 not in schedule:
            schedule[team1] = {}
        if team2 not in schedule:
            schedule[team2] = {}
        schedule[team1][week] = team2
        schedule[team2][week] = team1


In [73]:
players_rank = players_rank.sort_values(by='average_rank')
players_rank = players_rank[['player_id', 'Player', 'POS', 'average_rank']]


KeyError: 'average_rank'

In [46]:
schedule

{'Manager_1': {1: 'Manager_5',
  2: 'Manager_4',
  3: 'Manager_3',
  4: 'Manager_2',
  5: 'Manager_10',
  6: 'Manager_9',
  7: 'Manager_8',
  8: 'Manager_7',
  9: 'Manager_6',
  10: 'Manager_5',
  11: 'Manager_4',
  12: 'Manager_3',
  13: 'Manager_2'},
 'Manager_5': {1: 'Manager_1',
  2: 'Manager_9',
  3: 'Manager_9',
  4: 'Manager_9',
  5: 'Manager_9',
  6: 'Manager_10',
  7: 'Manager_10',
  8: 'Manager_10',
  9: 'Manager_10',
  10: 'Manager_1',
  11: 'Manager_9',
  12: 'Manager_9',
  13: 'Manager_9'},
 'Manager_10': {1: 'Manager_6',
  2: 'Manager_6',
  3: 'Manager_6',
  4: 'Manager_6',
  5: 'Manager_1',
  6: 'Manager_5',
  7: 'Manager_5',
  8: 'Manager_5',
  9: 'Manager_5',
  10: 'Manager_6',
  11: 'Manager_6',
  12: 'Manager_6',
  13: 'Manager_6'},
 'Manager_6': {1: 'Manager_10',
  2: 'Manager_10',
  3: 'Manager_10',
  4: 'Manager_10',
  5: 'Manager_2',
  6: 'Manager_2',
  7: 'Manager_2',
  8: 'Manager_2',
  9: 'Manager_1',
  10: 'Manager_10',
  11: 'Manager_10',
  12: 'Manager_10',

In [76]:
# Mergen Sie team_df und player_data auf 'Player ID' und 'Week'
results_df = pd.merge(team_df, player_data[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

# Gruppieren Sie das DataFrame nach Team und Woche und berechnen Sie die Gesamtpunktzahl für jedes Team in jeder Woche
results_df = results_df.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

# Benennen Sie die Spalte fantasy_points_ppr in Total Points um
results_df = results_df.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Sie können nun die Gesamtpunktzahl Ihres Teams in jeder Woche sehen
print(results_df[results_df['Team'] == 'Manager_1'])

# Sie können auch die Gesamtpunktzahl der gegnerischen Teams in jeder Woche sehen
print(results_df[results_df['Team'] != 'Manager_1'])


         Team  Week  Total Points
0   Manager_1     1         92.94
1   Manager_1     2         69.86
2   Manager_1     3         49.34
3   Manager_1     4         54.70
4   Manager_1     5         70.72
5   Manager_1     6         42.76
6   Manager_1     7         89.12
7   Manager_1     8         68.56
8   Manager_1     9         36.80
9   Manager_1    10         36.10
10  Manager_1    11         62.24
11  Manager_1    12         49.10
12  Manager_1    13         72.04
           Team  Week  Total Points
13   Manager_10     1        100.30
14   Manager_10     2         70.56
15   Manager_10     3        124.70
16   Manager_10     4         90.68
17   Manager_10     5         83.02
..          ...   ...           ...
125   Manager_9     9         63.20
126   Manager_9    10        112.08
127   Manager_9    11         84.84
128   Manager_9    12         59.74
129   Manager_9    13         87.88

[117 rows x 3 columns]


In [77]:
# Konvertieren Sie das dictionary zu einem DataFrame
schedule_df = pd.concat({k: pd.Series(v) for k, v in schedule.items()}).reset_index()

# Benennen Sie die Spalten um
schedule_df.columns = ['Team', 'Week', 'Opponent']

# Fügen Sie die Ergebnisse der Gegner in der gleichen Woche hinzu
results_with_opponents = pd.merge(results_df, schedule_df, how='left', on=['Team', 'Week'])

# Fügen Sie eine Spalte für die Punkte des Gegners hinzu
results_with_opponents['Opponent Points'] = results_with_opponents.apply(
    lambda row: results_df[
        (results_df['Team'] == row['Opponent']) &
        (results_df['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in results_df['Team'].values else 0,
    axis=1
)

# Fügen Sie eine Spalte hinzu, die anzeigt, ob das Team gewonnen hat
results_with_opponents['Won'] = results_with_opponents['Total Points'] > results_with_opponents['Opponent Points']

# Filtern Sie die Ergebnisse für Ihr Team
manager_1_results = results_with_opponents[results_with_opponents['Team'] == 'Manager_1']

# Zeigen Sie die Ergebnisse
print(manager_1_results[['Week', 'Total Points', 'Opponent', 'Opponent Points', 'Won']])


    Week  Total Points    Opponent  Opponent Points    Won
0      1         92.94   Manager_5            88.54   True
1      2         69.86   Manager_4            74.10  False
2      3         49.34   Manager_3            41.00   True
3      4         54.70   Manager_2            79.02  False
4      5         70.72  Manager_10            83.02  False
5      6         42.76   Manager_9            24.40   True
6      7         89.12   Manager_8            55.30   True
7      8         68.56   Manager_7            91.66  False
8      9         36.80   Manager_6            54.30  False
9     10         36.10   Manager_5            23.80   True
10    11         62.24   Manager_4            81.80  False
11    12         49.10   Manager_3            30.10   True
12    13         72.04   Manager_2            55.20   True


In [26]:
from gurobipy import *

# Liste der Spieler und Teams erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))
# Angenommen, jedes Team wählt 15 Spieler
num_players_per_team = 16
rounds = list(range(1, num_players_per_team+1))

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['average_rank'] for index, row in players_rank.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t] ist 1, wenn Spieler i in Woche j von Team t ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# z[t][r] ist 1, wenn Team t in Runde r einen Spieler auswählt
z = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# Zielfunktion: Maximiere die Gesamtpunktzahl für Manager_1 über alle Wochen
m.setObjective(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players for j in weeks), GRB.MAXIMIZE)

# Spielplan-Nebenbedingungen
for j in weeks:
    opponent = schedule['Manager_1'][j]
    m.addConstr(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players) >= quicksum(x[i, j, opponent] * expected_points_dict[(i, j)] for i in players))

for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players) == 9)
# Nebenbedingungen hinzufügen
# Ändern Sie die Einschränkungen für die Anzahl der Spieler pro Position
for t in teams:
    for j in weeks:
        # Es sollte einen QB geben
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'QB') == 1)
        # Es sollten mindestens 2 WRs und insgesamt 3 Spieler aus WR, RB oder TE (ohne den speziellen TE) geben
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] in ['WR', 'RB']) + 
                    quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 3)
        # Es sollte mindestens 2 RBs geben
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'RB') >= 2)
        # Es sollte mindestens einen TE geben
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'K') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'DST') == 2)



# Ein Spieler kann nur in einer Woche von einem Team ausgewählt werden
for i in players:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for t in teams) <= 1)

# Wenn ein Spieler von einem Team in einer Woche ausgewählt wird, muss er für die gesamte Saison ausgewählt sein
for i in players:
    for t in teams:
        m.addConstr(y[i, t] >= quicksum(x[i, j, t] for j in weeks) / len(weeks))

# Ein Spieler kann nur von einem Team für die gesamte Saison ausgewählt werden
for i in players:
    m.addConstr(quicksum(y[i, t] for t in teams) <= 1)

# Für Manager außer Manager_1, wähle den Spieler mit dem besten verfügbaren Ranking
# Für Manager außer Manager_1, wähle die Spieler basierend auf dem saisonalen Ranking
for t in teams:
    if t != 'Manager_1':
        #m.addConstr(quicksum(y[i, t] * season_ranking[i] for i in players) <= len(players))
        m.addConstr(quicksum(y[i, t] * season_ranking[i] for i in players if i in season_ranking and (i, t) in y) <= len(players))



# Jedes Team kann nur eine bestimmte Anzahl von Spielern pro Saison auswählen
for t in teams:
    m.addConstr(quicksum(y[i, t] for i in players) <= num_players_per_team)

# Draft-Reihenfolge-Nebenbedingungen
# Draft-Reihenfolge-Nebenbedingungen
for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(z[t, r] == 1)

# Problem lösen
m.optimize()

# Ausgabe der Ergebnisse
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

print('Total points: %g' % m.objVal)

# Extrahieren der ausgewählten Spieler für jedes Team und jede Woche
selected_players = [(i, j, t) for i in players for j in weeks for t in teams if x[i, j, t].x == 1]

# Umwandlung der Liste in ein DataFrame für eine bessere Darstellung
team_df = pd.DataFrame(selected_players, columns=['Player ID', 'Week', 'Team'])

team_df


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11878 rows, 63020 columns and 282886 nonzeros
Model fingerprint: 0x8b05bb4b
Variable types: 0 continuous, 63020 integer (63020 binary)
Coefficient statistics:
  Matrix range     [1e-02, 7e+02]
  Objective range  [1e-02, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 4480 rows and 23317 columns
Presolve time: 0.98s
Presolved: 7398 rows, 39703 columns, 138747 nonzeros
Variable types: 0 continuous, 39703 integer (39703 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.10s

Solved with primal simplex
Extra simplex iterations after uncrush: 1

Root relaxation: objective 2.794880e+03, 40697 iterations, 1.49 seconds (1.87 work units)

    Nodes    |    Current Node    |     Objective Bounds      |  

,Player ID,Week,Team
0,00-0022924,1,Manager_7
1,00-0022924,2,Manager_7
2,00-0022924,3,Manager_7
3,00-0022924,4,Manager_7
4,00-0022924,5,Manager_7
...,...,...,...
1165,00-000023,6,Manager_1
1166,00-000023,7,Manager_1
1167,00-000023,11,Manager_1
1168,00-000023,12,Manager_1


In [74]:
from gurobipy import *

# Liste der Spieler und Teams erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))

# Angenommen, jedes Team wählt 15 Spieler
num_players_per_team = 16
rounds = list(range(1, num_players_per_team+1))

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['average_rank'] for index, row in players_rank.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t] ist 1, wenn Spieler i in Woche j von Team t ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# z[t][r] ist 1, wenn Team t in Runde r einen Spieler auswählt
z = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][j] ist 1, wenn Team t in Woche j gewinnt
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Zielfunktion: Maximiere die Anzahl der Siege von Manager_1
m.setObjective(quicksum(w['Manager_1', j] for j in weeks), GRB.MAXIMIZE)

# Spielplan-Nebenbedingungen
for j in weeks:
    opponent = schedule['Manager_1'][j]
    m.addConstr(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players) - 
                quicksum(x[i, j, opponent] * expected_points_dict[(i, j)] for i in players) >= 
                w['Manager_1', j] - (1 - w[opponent, j]))

for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players) == 9)
# Nebenbedingungen hinzufügen
# Ändern Sie die Einschränkungen für die Anzahl der Spieler pro Position
for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'QB') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] in ['WR', 'RB']) + 
                    quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 3)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'RB') >= 2)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'K') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'DST') == 1)

for i in players:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for t in teams) <= 1)

for i in players:
    for t in teams:
        m.addConstr(y[i, t] >= quicksum(x[i, j, t] for j in weeks) / len(weeks))

for i in players:
    m.addConstr(quicksum(y[i, t] for t in teams) <= 1)

for t in teams:
    if t != 'Manager_1':
        m.addConstr(quicksum(y[i, t] * season_ranking[i] for i in players if i in season_ranking and (i, t) in y) <= len(players))

for t in teams:
    m.addConstr(quicksum(y[i, t] for i in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(z[t, r] == 1)

# Problem lösen
m.optimize()

# Ausgabe der Ergebnisse
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

print('Total wins: %g' % m.objVal)

# Extrahieren der ausgewählten Spieler für jedes Team und jede Woche
selected_players = [(i, j, t) for i in players for j in weeks for t in teams if x[i, j, t].x == 1]

# Umwandlung der Liste in ein DataFrame für eine bessere Darstellung
team_df = pd.DataFrame(selected_players, columns=['Player ID', 'Week', 'Team'])

team_df


KeyError: 'average_rank'

In [75]:
from gurobipy import *

# Liste der Spieler und Teams erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))

# Angenommen, jedes Team wählt 15 Spieler
num_players_per_team = 16
rounds = list(range(1, num_players_per_team+1))

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['rank'] for index, row in players_rank.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t] ist 1, wenn Spieler i in Woche j von Team t ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# z[t][r] ist 1, wenn Team t in Runde r einen Spieler auswählt
z = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][j] ist 1, wenn Team t in Woche j gewinnt
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Zielfunktion: Maximiere die Anzahl der Siege von Manager_1
m.setObjective(quicksum(w['Manager_1', j] for j in weeks), GRB.MAXIMIZE)

# Spielplan-Nebenbedingungen
for j in weeks:
    opponent = schedule['Manager_1'][j]
    m.addConstr(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players) - 
                quicksum(x[i, j, opponent] * expected_points_dict[(i, j)] for i in players) >= 
                w['Manager_1', j] - (1 - w[opponent, j]))

for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players) == 9)
# Nebenbedingungen hinzufügen
# Ändern Sie die Einschränkungen für die Anzahl der Spieler pro Position
for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'QB') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] in ['WR', 'RB']) + 
                    quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 3)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'RB') >= 2)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'K') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'DST') == 1)

for i in players:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for t in teams) <= 1)

for i in players:
    for t in teams:
        m.addConstr(y[i, t] >= quicksum(x[i, j, t] for j in weeks) / len(weeks))

for i in players:
    m.addConstr(quicksum(y[i, t] for t in teams) <= 1)

for t in teams:
    if t != 'Manager_1':
        m.addConstr(quicksum(y[i, t] * season_ranking[i] for i in players if i in season_ranking and (i, t) in y) <= len(players))

for t in teams:
    m.addConstr(quicksum(y[i, t] for i in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(z[t, r] == 1)

# Problem lösen
m.optimize()

# Ausgabe der Ergebnisse
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

print('Total wins: %g' % m.objVal)

# Extrahieren der ausgewählten Spieler für jedes Team und jede Woche
selected_players = [(i, j, t) for i in players for j in weeks for t in teams if x[i, j, t].x == 1]

# Umwandlung der Liste in ein DataFrame für eine bessere Darstellung
team_df = pd.DataFrame(selected_players, columns=['Player ID', 'Week', 'Team'])

team_df


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11878 rows, 63150 columns and 282922 nonzeros
Model fingerprint: 0xabb73d7d
Variable types: 0 continuous, 63150 integer (63150 binary)
Coefficient statistics:
  Matrix range     [8e-02, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 1003 rows and 4183 columns
Presolve time: 0.96s
Presolved: 10875 rows, 58967 columns, 210558 nonzeros
Variable types: 0 continuous, 58967 integer (58967 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.08s

Solved with primal simplex

Root relaxation: objective 1.300000e+01, 9937 iterations, 0.38 seconds (0.47 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

,Player ID,Week,Team
0,00-0019596,2,Manager_1
1,00-0019596,3,Manager_1
2,00-0019596,4,Manager_1
3,00-0019596,5,Manager_1
4,00-0019596,6,Manager_1
...,...,...,...
1165,00-000010,10,Manager_7
1166,00-000010,11,Manager_7
1167,00-000010,12,Manager_7
1168,00-000010,13,Manager_7


In [60]:
from gurobipy import *

# Liste der Spieler und Teams erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))

# Angenommen, jedes Team wählt 15 Spieler
num_players_per_team = 10
rounds = list(range(1, num_players_per_team+1))
num_weeks = 14

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['average_rank'] for index, row in players_rank.iterrows()}

# Erstelle ein dictionary, das die erwarteten Punkte für jeden Spieler pro Woche speichert
expected_points = {(row['player_id'], row['week']): row['predicted_fantasy_points'] for index, row in player_data.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t] ist 1, wenn Spieler i in Woche j von Team t ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# z[t][r] ist 1, wenn Team t in Runde r einen Spieler auswählt
z = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# Zielfunktion: Maximiere die Gesamtpunktzahl für Manager_1 über alle Wochen
m.setObjective(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players for j in weeks), GRB.MAXIMIZE)

# Spielplan-Nebenbedingungen
for j in weeks:
    opponent = schedule['Manager_1'][j]
    m.addConstr(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players) >= quicksum(x[i, j, opponent] * expected_points_dict[(i, j)] for i in players))

# Positionen-Nebenbedingungen
for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'QB') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] in ['RB', 'WR']) >= 5)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'K') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'DST') == 1)

# Ein Spieler kann nur in einer Woche von einem Team ausgewählt werden
for i in players:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for t in teams) <= 1)

# Ein Spieler kann nur in einer Woche aufgestellt werden, wenn er vom Team ausgewählt wurde
for i in players:
    for j in weeks:
        for t in teams:
            m.addConstr(x[i, j, t] <= y[i, t])

# Jedes Team wählt genau num_players_per_team Spieler aus
for t in teams:
    m.addConstr(quicksum(y[i, t] for i in players) == num_players_per_team)

# Für Manager außer Manager_1, wähle die Spieler basierend auf dem saisonalen Ranking und den erwarteten Punkten
for t in teams:
    if t != 'Manager_1':
        m.addConstr(quicksum(y[i, t] * ((1/season_ranking[i]) + quicksum(expected_points[(i, j)] for j in weeks if (i, j) in expected_points)) for i in players if i in season_ranking and (i, t) in y) <= len(players) * num_weeks)

m.optimize()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 64889 rows, 62960 columns and 249056 nonzeros
Model fingerprint: 0x96106abe
Variable types: 0 continuous, 62960 integer (62960 binary)
Coefficient statistics:
  Matrix range     [2e-03, 2e+02]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]
Presolve removed 9 rows and 100 columns
Presolve time: 0.29s
Presolved: 64880 rows, 62860 columns, 245422 nonzeros
Variable types: 0 continuous, 62860 integer (62860 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 1.06s

Solved with primal simplex

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   76064    1.8288103e+03   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 1.828810e+03, 76064 itera

In [48]:
from gurobipy import *
import numpy as np

# Liste der Spieler und Teams erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))
# Angenommen, jedes Team wählt 15 Spieler
num_players_per_team = 10
rounds = list(range(1, num_players_per_team+1))

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['average_rank'] for index, row in players_rank.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t] ist 1, wenn Spieler i in Woche j von Team t ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[i, r] ist 1, wenn Spieler i in Runde r ausgewählt wird.
d = m.addVars(players, rounds, vtype=GRB.BINARY, name="drafted_by_round")

# Anfangsspielerliste
available_players = players.copy()

# Durchlaufen Sie die Runden sequenziell
for r in rounds:
    # Zielfunktion: Maximiere die Gesamtpunktzahl für Manager_1 über alle Wochen
    m.setObjective(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in available_players for j in weeks), GRB.MAXIMIZE)

    # Spielplan-Nebenbedingungen
    for j in weeks:
        opponent = schedule['Manager_1'][j]
        m.addConstr(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players) >= quicksum(x[i, j, opponent] * expected_points_dict[(i, j)] for i in players))

    for t in teams:
        for j in weeks:
            m.addConstr(quicksum(x[i, j, t] for i in players) == 9)

    # Nebenbedingungen hinzufügen
    for t in teams:
        for j in weeks:
            m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'QB') == 1)
            m.addConstr(quicksum(x[i, j, t] for i in players if position[i] in ['WR', 'RB']) + 
                        quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 3)
            m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'RB') >= 2)
            m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 1)
            m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'K') == 1)
            m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'DST') == 2)

    # Ein Spieler kann nur in einer Woche von einem Team ausgewählt werden
    for i in players:
        for j in weeks:
            m.addConstr(quicksum(x[i, j, t] for t in teams) <= 1)

    # Wenn ein Spieler von einem Team in einer Woche ausgewählt wird, muss er für die gesamte Saison ausgewählt sein
    for i in players:
        for t in teams:
            m.addConstr(y[i, t] >= quicksum(x[i, j, t] for j in weeks) / len(weeks))

    # Ein Spieler kann nur von einem Team für die gesamte Saison ausgewählt werden
    for i in players:
        m.addConstr(quicksum(y[i, t] for t in teams) <= 1)

    # Jedes Team kann nur eine bestimmte Anzahl von Spielern pro Saison auswählen
    for t in teams:
        m.addConstr(quicksum(y[i, t] for i in players) <= num_players_per_team)

    # Nebenbedingung, um sicherzustellen, dass ein Spieler in genau einer Runde ausgewählt wird.
    for i in players:
        m.addConstr(quicksum(d[i, r] for r in rounds) == quicksum(y[i, t] for t in teams))
        
    # Nur in der aktuellen Runde werden Spieler ausgewählt
    m.addConstr(quicksum(d[i, r] for i in available_players) == len(teams))

    # Die Auswahl in jeder Runde sollte basierend auf dem Ranking sein.
    max_rank = max(season_ranking.values())
    
    for t in teams:
        if t != 'Manager_1':
            # Spieler mit niedrigstem Rang aus den verfügbaren Spielern auswählen
            min_rank_player = min(available_players, key=lambda p: season_ranking.get(p, max_rank+1))
            # Weisen Sie diesen Spieler diesem Team zu
            m.addConstr(d[min_rank_player, r] == 1)
            # Entfernen Sie diesen Spieler aus der Liste der verfügbaren Spieler
            available_players.remove(min_rank_player)

    # Problem lösen
    m.optimize()

# Ausgabe der Ergebnisse
#for v in m.getVars():
    #if v.x > 0:
        #p#rint('%s : %g' % (v.varName, v.x))

#print('Total points: %g' % m.objVal)

# Extrahieren der ausgewählten Spieler für jedes Team und jede Woche
selected_players = [(i, j, t) for i in players for j in weeks for t in teams if x[i, j, t].x == 1]

# Umwandlung der Liste in ein DataFrame für eine bessere Darstellung
team_df = pd.DataFrame(selected_players, columns=['Player ID', 'Week', 'Team'])

team_df


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12168 rows, 67350 columns and 288546 nonzeros
Model fingerprint: 0xc676728c
Variable types: 0 continuous, 67350 integer (67350 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+01]
  Objective range  [1e-02, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 148 rows and 3610 columns
Presolve time: 0.23s
Presolved: 12020 rows, 63740 columns, 226446 nonzeros
Variable types: 0 continuous, 63740 integer (63740 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.02s

Solved with dual simplex

Root relaxation: objective 2.835971e+03, 5594 iterations, 0.22 seconds (0.15 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | In

     0     0 2117.83930    0 1860          - 2117.83930      -     -   13s
     0     0 2110.74345    0 1934          - 2110.74345      -     -   15s


In [61]:
# Ausgabe der ausgewählten Spieler für jedes Team
for t in teams:
    print(f'Team {t} hat folgende Spieler ausgewählt:')
    for i in players:
        if y[i, t].x > 0:
            print(f'Player ID: {i}')


Team Manager_10 hat folgende Spieler ausgewählt:
Player ID: 00-0019596
Player ID: 00-0022921
Player ID: 00-0022943
Player ID: 00-0032897
Player ID: 00-0032918
Player ID: 00-0033293
Player ID: 00-0033296
Player ID: 00-0034844
Player ID: 00-0029421
Player ID: 00-000032
Team Manager_2 hat folgende Spieler ausgewählt:
Player ID: 00-0020531
Player ID: 00-0023500
Player ID: 00-0025399
Player ID: 00-0030108
Player ID: 00-0032112
Player ID: 00-0032940
Player ID: 00-0032951
Player ID: 00-0033681
Player ID: 00-0025580
Player ID: 00-000008
Team Manager_4 hat folgende Spieler ausgewählt:
Player ID: 00-0021206
Player ID: 00-0022924
Player ID: 00-0032227
Player ID: 00-0033567
Player ID: 00-0033789
Player ID: 00-0033856
Player ID: 00-0033893
Player ID: 00-0034676
Player ID: 00-0023252
Player ID: 00-000026
Team Manager_6 hat folgende Spieler ausgewählt:
Player ID: 00-0022787
Player ID: 00-0022942
Player ID: 00-0028097
Player ID: 00-0033839
Player ID: 00-0033891
Player ID: 00-0033921
Player ID: 00-0033

In [62]:
# Liste für die ausgewählten Spieler-Daten
selected_players_data = []

# Füllen Sie die Liste mit den ausgewählten Spielern für jedes Team
for t in teams:
    for i in players:
        if y[i, t].x > 0:
            selected_players_data.append({
                'Team': t,
                'Player ID': i,
                'Player Name': player_data.loc[player_data['player_id'] == i, 'player_display_name'].iloc[0],
                'Position': player_data.loc[player_data['player_id'] == i, 'position_group'].iloc[0]
            })

# Erstellen Sie ein DataFrame aus der Liste
selected_players_df = pd.DataFrame(selected_players_data)

selected_players_df


,Team,Player ID,Player Name,Position
0,Manager_10,00-0019596,Tom Brady,QB
1,Manager_10,00-0022921,Larry Fitzgerald,WR
2,Manager_10,00-0022943,Benjamin Watson,TE
3,Manager_10,00-0032897,Derek Watt,RB
4,Manager_10,00-0032918,C.J. Ham,RB
...,...,...,...,...
95,Manager_9,00-0033871,Corey Davis,WR
96,Manager_9,00-0033873,Patrick Mahomes,QB
97,Manager_9,00-0033908,Cooper Kupp,WR
98,Manager_9,00-0032569,Wil Lutz,K


In [63]:
# Berechnen Sie die Punkte für jedes Spiel
for j in weeks:
    print(f'Week {j}:')
    for t in teams:
        points = sum(x[i, j, t].x * expected_points_dict[(i, j)] for i in players)
        print(f'Team {t} scored {points} points.')


Week 1:
Team Manager_10 scored 72.3193923 points.
Team Manager_2 scored 51.1560865 points.
Team Manager_4 scored 56.9596646 points.
Team Manager_6 scored 64.38693359999999 points.
Team Manager_8 scored 82.26866826999999 points.
Team Manager_1 scored 163.41949849999997 points.
Team Manager_3 scored 71.1532146 points.
Team Manager_5 scored 47.17168769999999 points.
Team Manager_7 scored 76.95295639999999 points.
Team Manager_9 scored 84.3931935 points.
Week 2:
Team Manager_10 scored 59.078646100000014 points.
Team Manager_2 scored 50.64079 points.
Team Manager_4 scored 64.6040129 points.
Team Manager_6 scored 71.83213520000001 points.
Team Manager_8 scored 81.4065531 points.
Team Manager_1 scored 158.71189300000003 points.
Team Manager_3 scored 80.2494883 points.
Team Manager_5 scored 48.1998964 points.
Team Manager_7 scored 71.755269 points.
Team Manager_9 scored 85.851023 points.
Week 3:
Team Manager_10 scored 58.0915114 points.
Team Manager_2 scored 37.2581175 points.
Team Manager_4 s

In [70]:
from gurobipy import *

# Liste der Spieler, Teams und Wochen erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))
positions = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "FLEX": 1}

# Angenommen, jedes Team wählt 10 Spieler
num_players_per_team = 10
rounds = list(range(1, num_players_per_team+1))

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['rank'] for index, row in players_rank.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t][p] ist 1, wenn Spieler i in Woche j von Team t für Position p ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, positions.keys(), vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# z[t][r] ist 1, wenn Team t in Runde r einen Spieler auswählt
z = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# Zielfunktion: Maximiere die Gesamtpunktzahl für Manager_1 über alle Wochen
m.setObjective(quicksum(x[i, j, 'Manager_1', p] * expected_points_dict[(i, j)] for i in players for j in weeks for p in positions.keys()), GRB.MAXIMIZE)

# Spielplan-Nebenbedingungen
for j in weeks:
    opponent = schedule['Manager_1'][j]
    m.addConstr(quicksum(x[i, j, 'Manager_1', p] * expected_points_dict[(i, j)] for i in players for p in positions.keys()) >= quicksum(x[i, j, opponent, p] * expected_points_dict[(i, j)] for i in players for p in positions.keys()))

# Position Constraints
for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t, 'QB'] for i in players if position[i] == 'QB') == 1)
        m.addConstr(quicksum(x[i, j, t, 'WR'] for i in players if position[i] == 'WR') >= 2)
        m.addConstr(quicksum(x[i, j, t, 'RB'] for i in players if position[i] == 'RB') >= 2)
        m.addConstr(quicksum(x[i, j, t, 'TE'] for i in players if position[i] == 'TE') >= 1)
        m.addConstr(quicksum(x[i, j, t, 'FLEX'] for i in players if position[i] in ['RB', 'WR', 'TE']) == 1)
        m.addConstr(quicksum(x[i, j, t, p] for i in players for p in positions.keys()) == 7)

# Ein Spieler kann nur in einer Woche von einem Team ausgewählt werden
for i in players:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t, p] for t in teams for p in positions.keys()) <= 1)

# Wenn ein Spieler von einem Team in einer Woche ausgewählt wird, muss er für die gesamte Saison ausgewählt sein
for i in players:
    for t in teams:
        m.addConstr(y[i, t] >= quicksum(x[i, j, t, p] for j in weeks for p in positions.keys()) / len(weeks))

# Ein Spieler kann nur von einem Team für die gesamte Saison ausgewählt werden
for i in players:
    m.addConstr(quicksum(y[i, t] for t in teams) <= 1)

# Jedes Team kann nur eine bestimmte Anzahl von Spielern pro Saison auswählen
for t in teams:
    m.addConstr(quicksum(y[i, t] for i in players) <= num_players_per_team)

# Problem lösen
m.optimize()

# Ausgabe der Ergebnisse
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

print('Total points: %g' % m.objVal)


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13931 rows, 361120 columns and 1252050 nonzeros
Model fingerprint: 0x475e14ca
Variable types: 0 continuous, 361120 integer (361120 binary)
Coefficient statistics:
  Matrix range     [8e-02, 4e+01]
  Objective range  [3e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 0 rows and 213430 columns
Presolve time: 1.19s
Presolved: 13931 rows, 147690 columns, 519506 nonzeros
Variable types: 0 continuous, 147690 integer (147690 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.12s

Barrier performed 0 iterations in 2.19 seconds (3.90 work units)
Barrier solve interrupted - model solved by another algorithm

Concurrent spin time: 0.00s

Solved with dual simplex

Roo

In [77]:
# Aufteilen der ersten Spalte in zwei separate Spalten
team_points_df[['Team', 'Week']] = pd.DataFrame(team_points_df['index'].tolist(), index=team_points_df.index)

# Löschen der ursprünglichen 'index' Spalte
team_points_df.drop(columns=['index'], inplace=True)

# Umbenennen der Spalten
team_points_df.columns = ['Points', 'Team', 'Week']

# Sortiere das DataFrame nach Team und Woche
team_points_df.sort_values(['Team', 'Week'], inplace=True)

print(team_points_df)


         Points       Team  Week
65   101.081543  Manager_1     1
66   109.029633  Manager_1     2
67   120.006226  Manager_1     3
68   129.499832  Manager_1     4
69   115.601074  Manager_1     5
..          ...        ...   ...
125   39.973007  Manager_9     9
126   40.476349  Manager_9    10
127   37.236725  Manager_9    11
128   65.149582  Manager_9    12
129   71.557724  Manager_9    13

[130 rows x 3 columns]


In [75]:
# Extrahieren der ausgewählten Spieler für jedes Team und jede Woche
selected_players = [(i, j, t, p) for i in players for j in weeks for t in teams for p in positions.keys() if x[i, j, t, p].x == 1]

# Initialisiere ein Wörterbuch, um die Punktzahlen zu speichern
team_points = {t: {j: 0 for j in weeks} for t in teams}

# Gehe durch die ausgewählten Spieler und summiere ihre erwarteten Punkte für jedes Team in jeder Woche
for i, j, t, p in selected_players:
    team_points[t][j] += expected_points_dict[(i, j)]

# Erstelle ein DataFrame aus dem Wörterbuch
team_points_df = pd.DataFrame.from_dict({(t,j): team_points[t][j] 
                                         for t in team_points.keys() 
                                         for j in team_points[t].keys()},
                                        orient='index')

# Umbenennen der Spalten
team_points_df.reset_index(inplace=True)
team_points_df.columns = ['Team', 'Week', 'Points']

# Sortiere das DataFrame nach Team und Woche
team_points_df.sort_values(['Team', 'Week'], inplace=True)

# Berechne die Gesamtpunkte für jedes Team über die Saison
total_points = team_points_df.groupby('Team')['Points'].sum()

# Zeige das DataFrame
print(team_points_df)
print(total_points)


ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements

In [40]:
from gurobipy import Model, GRB, quicksum
import numpy as np 

def optimize_fantasy_football_gurobi():
    # Ihr Optimierungscode hier...



    # Anfangszustand setzen
    player_data['predicted_fantasy_points'].fillna(0, inplace=True)

    weeks = range(1, 18)
    managers = ['Manager' + str(i) for i in range(1, 11)]  # 10 Manager

    # Ein Dictionary mit dem Team jedes Spielers erstellen
    player_teams = dict(zip(player_data['player_id'], player_data['recent_team']))

    nfl_teams = clean_date['home_team'].unique().tolist()

    # Die Anzahl der Draft-Picks festlegen (z.B. Anzahl der Manager * Anzahl der Spieler pro Team)
    nD = len(managers) * 5  # anpassen entsprechend der Anzahl der Spieler pro Team

    # Erstellen Sie ein Dictionary, das jedem Spieler-Team-Kombination die Stärke des Gegners zuweist
    opponent_strength = {}
    for _, row in clean_date.iterrows():
        # Fügen Sie die Punkte für das Heimteam hinzu
        opponent_strength[(row['week'], row['home_team'])] = row['away_score']
        # Fügen Sie die Punkte für das Auswärtsteam hinzu
        opponent_strength[(row['week'], row['away_team'])] = row['home_score']
    # Eine Liste aller einzigartigen Spieler erstellen
    players = player_data['player_id'].unique()

    # Werte für alle Teams und Wochen festlegen
    for team in nfl_teams:
        for week in weeks:
            if (week, team) not in opponent_strength:
                # Wenn das Team in dieser Woche eine Bye Week hat, 
                # setzen Sie die Stärke des Gegners auf einen hohen Wert
                opponent_strength[(week, team)] = 1e10

    # Ein Dictionary mit den vorhergesagten Punkten für jeden Spieler in jeder Woche erstellen
    predicted_points = {(player, week): points for player, week, points in zip(player_data['player_id'], player_data['week'], player_data['predicted_fantasy_points'])}

    # Ein Dictionary mit der Position jedes Spielers erstellen
    position = dict(zip(player_data['player_id'], player_data['position_group']))

    # Die Anzahl der Draft-Picks festlegen (z.B. Anzahl der Manager * Anzahl der Spieler pro Team)
    nD = len(managers) * 5  # anpassen entsprechend der Anzahl der Spieler pro Team

    # Die Auswahl der Spieler für jeden Draft-Pick
    DMPlayer = np.empty((nD+1, len(players)), dtype=object)

    

    # Iteration über die gesamte Anzahl der Draft-Picks
    for k in range(1, nD+1):
        # Problem-Objekt erstellen
        model = Model("FantasyFootball")
        
        # Entscheidungsvariablen definieren
        players_vars = model.addVars(((i, j) for i in players for j in managers), vtype=GRB.BINARY, name="Players")
        
        # Definieren Sie die Siege von Manager_1 als binäre Variablen
        wins = model.addVars((week for week in weeks), vtype=GRB.BINARY, name="Wins")

        # Zielfunktion definieren
        model.setObjective(quicksum(players_vars[i,'Manager1']*predicted_points[(i,week)] / (opponent_strength[(week, player_teams[i])] + 1e-10) for i in players for week in weeks if (i, week) in predicted_points) + quicksum(wins[week] for week in weeks), GRB.MAXIMIZE)




        for week in weeks:
            model.addConstr(quicksum([players_vars[i,'Manager1']*predicted_points[(i,week)] / (opponent_strength[(week, player_teams[i])] + 1e-10) for i in players if (i, week) in predicted_points]) >= quicksum([players_vars[i,j]*predicted_points[(i,week)] / (opponent_strength[(week, player_teams[i])] + 1e-10) for i in players for j in ['Manager2', 'Manager3'] if (i, week) in predicted_points]) + (1 - wins[week])*100000)
            model.addConstr(quicksum([players_vars[i,'Manager1']*predicted_points[(i,week)] / (opponent_strength[(week, player_teams[i])] + 1e-10) for i in players if (i, week) in predicted_points]) <= quicksum([players_vars[i,j]*predicted_points[(i,week)] / (opponent_strength[(week, player_teams[i])] + 1e-10) for i in players for j in ['Manager2', 'Manager3'] if (i, week) in predicted_points]) + wins[week]*100000)

        # Positionseinschränkungen für jedes Team definieren
        for manager in managers:
            model.addConstr(quicksum([players_vars[i,manager] for i in players if position[i] == 'QB']) == 1)
            model.addConstr(quicksum([players_vars[i,manager] for i in players if position[i] == 'WR']) == 2)
            model.addConstr(quicksum([players_vars[i,manager] for i in players if position[i] == 'RB']) == 2)
            model.addConstr(quicksum([players_vars[i,manager] for i in players if position[i] == 'TE']) == 1)

        # Einschränkung definieren, dass ein Spieler nur einmal gedraftet werden kann
        for i in players:
            model.addConstr(quicksum([players_vars[i,j] for j in managers]) <= 1)

        # Problem lösen
        model.optimize()

        # Finden Sie den neu ausgewählten Spieler
        new_player = max([v for v in model.getVars() if "Players" in v.varName and v.x > 0], key=lambda v: v.x)

        # Der Spielername ist der Teil der Variablennamen nach "Players["
        player_name_raw = new_player.varName.split("[", 1)[1]

        # Entfernen Sie die unerwünschten Zeichen
        player_name = player_name_raw.replace("('", "").replace("',_'Manager1')", "")

        DMPlayer[k] = player_name

        # Aktualisieren Sie das Optimierungsproblem, um den neu ausgewählten Spieler aus der verfügbaren Spielerliste zu entfernen
        for i in players:
            new_player_id = new_player.varName.split("[", 1)[1].replace("('", "").replace("',_'Manager1')", "")
            if i == new_player_id:
                model.addConstr(players_vars[i, 'Manager1'] == 0)

    # Ausgabe der Ergebnisse
    for v in model.getVars():
        if v.x > 0:
            print(v.varName, "=", v.x)
    # Am Ende der Funktion:
    return model, weeks, predicted_points  # Fügen Sie diese Zeile am Ende Ihrer Funktion hinzu

# Dann rufen Sie Ihre Funktion auf diese Weise auf:
# Dann rufen Sie Ihre Funktion auf diese Weise auf:
model, weeks, predicted_points = optimize_fantasy_football_gurobi()

# Nun können Sie auf diese Variablen zugreifen:
print("Team von Manager1:")
players_vars = {}
for v in model.getVars():
    if v.x > 0 and "Manager1" in v.varName and "Players" in v.varName:
        # Der Spielername ist der Teil der Variablennamen nach "Players["
        player_name_raw = v.varName.split("[", 1)[1]
        # Entfernen Sie die unerwünschten Zeichen
        player_name = player_name_raw.split(",", 1)[0].replace("('", "").replace("'", "")
        players_vars[player_name] = v

# Ausgabe der Gesamtpunktzahl für Manager1
print("Ergebnisse für Manager1:")
total_points = 0
for player in players_vars:
    for week in weeks:
        if (player, week) in predicted_points:
            points = predicted_points[(player, week)]
            print(f"   Spieler {player} in Woche {week}: {points} Punkte")
            total_points += points
print(f"   Gesamtpunktzahl: {total_points}")
# Und dann können Sie auf das Modell zugreifen:
print("Team von Manager1:")
for v in model.getVars():
    if v.x > 0 and "_Manager1" in v.varName:
        # Der Spielername ist der Teil der Variablennamen nach "Players["
        player_name_raw = v.varName.split("[", 1)[1]
        # Entfernen Sie die unerwünschten Zeichen
        player_name = player_name_raw.replace("('", "").replace("',_'Manager1')", "")
        print(player_name)
optimize_fantasy_football_gurobi()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 621 rows, 5487 columns and 39738 nonzeros
Model fingerprint: 0x01c7da91
Variable types: 0 continuous, 5487 integer (5487 binary)
Coefficient statistics:
  Matrix range     [5e-10, 2e+11]
  Objective range  [1e-01, 3e+11]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 66.7403696
Presolve removed 8 rows and 11 columns
Presolve time: 0.03s
Presolved: 613 rows, 5476 columns, 25334 nonzeros
Variable types: 6 continuous, 5470 integer (5470 binary)
Found heuristic solution: objective 74.5488331

Root relaxation: objective 1.110959e+12, 378 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wo

(<gurobi.Model MIP instance FantasyFootball: 621 constrs, 5487 vars, Parameter changes: Username=(user-defined)>,
 range(1, 18),
 {('00-0019596', 1): 17.675735473632812,
  ('00-0019596', 2): 15.032180786132812,
  ('00-0019596', 3): 17.756942749023438,
  ('00-0019596', 4): 12.576644897460938,
  ('00-0019596', 5): 16.997833251953125,
  ('00-0019596', 6): 19.424072265625,
  ('00-0019596', 7): 16.303359985351562,
  ('00-0019596', 8): 16.203353881835938,
  ('00-0019596', 9): 17.672637939453125,
  ('00-0019596', 11): 16.617263793945312,
  ('00-0019596', 12): 11.880661010742188,
  ('00-0019596', 13): 18.405014038085938,
  ('00-0019596', 14): 15.20068359375,
  ('00-0019596', 15): 14.417404174804688,
  ('00-0019596', 16): 16.18939208984375,
  ('00-0019596', 17): 14.896591186523438,
  ('00-0020531', 1): 18.147415161132812,
  ('00-0020531', 2): 8.405654907226562,
  ('00-0020531', 8): 25.262924194335938,
  ('00-0020531', 10): 19.362869262695312,
  ('00-0020531', 11): 16.84423828125,
  ('00-0020531

In [49]:
def simulate_other_managers_picks(players, players_rank, num_picks):
  
    players_rank = players_rank.sort_values(by="rank")

   
    picked_players = players_rank.head(num_picks)

   
    players = players.loc[~players['player_id'].isin(picked_players['player_id'])]

    return players


In [67]:


opponent_pairs = [(team1, team2) for i, team1 in enumerate(teams) for team2 in teams[i+1:]]

player_data_2019 = player_data_2019.sort_values(by=["player_id", 'week'], ascending=False)

num_picks = 10  
picked_players = simulate_other_managers_picks(player_data_2019, players_rank, num_picks)

player_data_2019 = player_data_2019.loc[~player_data_2019['player_id'].isin(picked_players['player_id'])]

players = player_data_2019['player_id'].unique().tolist()
weeks = player_data_2019['week'].unique().tolist()
opponents = ["Manager1", "Manager2", "Manager3", "Manager4", "Manager5", "Manager6", "Manager7", "Manager8", "Manager9"]
position_groups = player_data_2019['position_group'].unique().tolist()

predicted_points = player_data_2019.set_index(['player_id', 'week'])['predicted_fantasy_points'].to_dict()

x = LpVariable.dicts("x", ((player, week) for player in players for week in weeks), cat='Binary')

for match, (team1, team2) in enumerate(opponent_pairs):
    for week in weeks:
        win[(team1, week, match)] = LpVariable(f"win_({team1},_{week},_{match})", lowBound=0, upBound=1, cat='Integer')
        win[(team2, week, match)] = LpVariable(f"win_({team2},_{week},_{match})", lowBound=0, upBound=1, cat='Integer')


valid_combinations = [key for key in predicted_points.keys()]

model += lpSum([x[(player, week)] * predicted_points[(player, week)] for (player, week) in valid_combinations]) \
    + lpSum([win[(opponent, week)] for opponent in opponents for week in weeks])

for team in teams:
    for week in weeks:
     
        model += lpSum([x[(player, week)] for (player, week) in valid_combinations if player in rosters[team]]) <= 11
        
        model += lpSum([x[(player, week)] for (player, week) in valid_combinations if player in rosters[team] and players_position[player] == "QB"]) <= 2
        
        model += lpSum([x[(player, week)] for (player, week) in valid_combinations if player in rosters[team] and players_position[player] == "RB"]) <= 5
 
        model += lpSum([x[(player, week)] for (player, week) in valid_combinations if player in rosters[team] and players_position[player] == "WR"]) <= 5
        
        model += lpSum([x[(player, week)] for (player, week) in valid_combinations if player in rosters[team] and players_position[player] == "TE"]) <= 2
    


for match, (team1, team2) in enumerate(opponent_pairs):
    for week in weeks:
        model += win[(team1, week, match)] + win[(team2, week, match)] <= 1
        model += lpSum([x[(player, week)] * predicted_points[(player, week)] for (player, week) in valid_combinations if player in rosters[team1]]) >= lpSum([x[(player, week)] * predicted_points[(player, week)] for (player, week) in valid_combinations if player in rosters[team2]]) - 1000*(1-win[(team1, week, match)])
        model += lpSum([x[(player, week)] * predicted_points[(player, week)] for (player, week) in valid_combinations if player in rosters[team2]]) >= lpSum([x[(player, week)] * predicted_points[(player, week)] for (player, week) in valid_combinations if player in rosters[team1]]) - 1000*(1-win[(team2, week, match)])
for plr in players:
    model += lpSum([x[(player, week)] for (player, week) in valid_combinations if player == plr]) <= 1  # Ein Spieler kann nur einmal ausgewählt werden

for opponent in opponents:
    for wk in weeks:
        model += win[(opponent, week)] <= lpSum([x[(player, week)] * predicted_points[(player, week)] for (player, week) in valid_combinations if week == wk])





status = model.solve(GLPK_CMD())

# Hole die Ergebnisse
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in x.values():
    print(f"{var.name}: {var.value()}")
for var in win.values():
    print(f"{var.name}: {var.value()}")


/Users/mariu/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


NameError: name 'rosters' is not defined

In [62]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# Erstelle das Model
model = LpProblem(name="fantasy-football-draft", sense=LpMaximize)

# Definiere die Sets
players = player_data_2019['player_id'].unique().tolist()
weeks = player_data_2019['week'].unique().tolist()
opponents = ["Manager1", "Manager2", "Manager3", "Manager4", "Manager5", "Manager6", "Manager7", "Manager8", "Manager9"]
position_groups = player_data_2019['position_group'].unique().tolist()

# Definiere die Parameter
predicted_points = player_data_2019.set_index(['player_id', 'week'])['predicted_fantasy_points'].to_dict()

# Definiere die Decision Variablen: x is binary
x = LpVariable.dicts("x", ((player, week) for player in players for week in weeks), cat='Binary')

win = LpVariable.dicts("win", ((opponent, week) for opponent in opponents for week in weeks), cat='Binary')


# Erzeuge eine Liste der Spieler/Wochen-Kombinationen, für die wir Vorhersagen haben
valid_combinations = [key for key in predicted_points.keys()]

# Zielfunktion
model += lpSum([x[(player, week)] * predicted_points[(player, week)] for (player, week) in valid_combinations]) \
    + lpSum([win[(opponent, week)] for opponent in opponents for week in weeks])

# Beschränkungen
for wk in weeks:
    model += lpSum([x[(player, week)] for (player, week) in valid_combinations if week == wk]) <= 9  # Teamgröße
    model += lpSum([x[(player, week)] for (player, week) in valid_combinations if week == wk and player_data_2019.loc[player_data_2019['player_id'] == player, 'position_group'].iloc[0] == 'QB']) <= 1  # QB Limit
    model += lpSum([x[(player, week)] for (player, week) in valid_combinations if week == wk and player_data_2019.loc[player_data_2019['player_id'] == player, 'position_group'].iloc[0] == 'RB']) <= 2  # RB Limit
    model += lpSum([x[(player, week)] for (player, week) in valid_combinations if week == wk and player_data_2019.loc[player_data_2019['player_id'] == player, 'position_group'].iloc[0] == 'WR']) <= 2  # WR Limit
    model += lpSum([x[(player, week)] for (player, week) in valid_combinations if week == wk and player_data_2019.loc[player_data_2019['player_id'] == player, 'position_group'].iloc[0] == 'TE']) <= 1  # TE Limit

for plr in players:
    model += lpSum([x[(player, week)] for (player, week) in valid_combinations if player == plr]) <= 1  # Ein Spieler kann nur einmal ausgewählt werden

for opponent in opponents:
    for wk in weeks:
        model += win[(opponent, week)] <= lpSum([x[(player, week)] * predicted_points[(player, week)] for (player, week) in valid_combinations if week == wk])




# Löse das Modell
status = model.solve()

# Hole die Ergebnisse
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in x.values():
    print(f"{var.name}: {var.value()}")
for var in win.values():
    print(f"{var.name}: {var.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mariu/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/5e47fe7730a646ba8e8e9f3f02ce04fc-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/5e47fe7730a646ba8e8e9f3f02ce04fc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 253 COLUMNS
At line 2996 RHS
At line 3245 BOUNDS
At line 3541 ENDATA
Problem MODEL has 248 rows, 295 columns and 1857 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 385.257 - 0.00 seconds
Cgl0004I processed model has 180 rows, 151 columns (151 integer (151 of which binary)) and 1684 elements
Cbc0038I Initial state - 33 integers unsatisfied sum - 11.8125
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -376.257 iterations 30
Cbc0038I Solut

In [60]:
# Lösen das Modell
model.solve()

# Überprüfe den Status der Lösung
print("Status:", LpStatus[model.status])

# Ausgabe der ausgewählten Spieler
for v in model.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mariu/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/d2dfab2c582f4fb4b3c2f8ec0fbd8723-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/d2dfab2c582f4fb4b3c2f8ec0fbd8723-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 253 COLUMNS
At line 29266 RHS
At line 29515 BOUNDS
At line 29811 ENDATA
Problem MODEL has 248 rows, 295 columns and 28127 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 182.779 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -182.779 to -1.79769e+308
Probing was tried 0 times and created 0 c

In [368]:
import numpy as np

positions = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'BN':2}
team = {}

for position, limit in positions.items():
    available_players = players[players['position_group'] == position]
    
    for i in range(limit):
        if len(available_players) > 0:
            top_player = available_players.iloc[0]
            team[top_player['player_id']] = top_player['position_group']
            players = players[players['player_id'] != top_player['player_id']]
        else:
            break

# Jetzt, füllen Sie die Bankpositionen auf 
available_players = players

for i in range(positions['BN']):
    if len(available_players) > 0:
        top_player = available_players.iloc[0]
        team[top_player['player_id']] = 'BN'
        players = players[players['player_id'] != top_player['player_id']]

print(team)


{'00-0033077': 'QB', '00-0032764': 'RB', '00-0034418': 'WR', '00-0034788': 'TE', '00-0033045': 'BN'}


In [31]:
from pulp import *
import pandas as pd
import numpy as np


all_players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()

position = dict(zip(player_data['player_id'], player_data['position_group']))
players_rank['position'] = players_rank['player_id'].apply(lambda x: position.get(x))

# Update players_rank DataFrame to include only available players
players_rank = players_rank[players_rank['player_id'].isin(all_players)]
position_limits = {'QB': 1, 'WR': 2, 'RB': 2, 'TE': 1}

# To keep track of players picked by each manager in each position
picked_players = {team: {pos: [] for pos in position_limits.keys()} for team in teams}

def pick_other_managers(players, players_rank):
    other_managers_picks = []
    position_counts = {'QB': 0, 'WR': 0, 'RB': 0, 'TE': 0}  # Counts for each position

    for _, player in players_rank.iterrows():
        if player['player_id'] in players:
            # Check if adding this player would violate the position restrictions
            if position_counts[player['position']] < position_limits[player['position']]:
                other_managers_picks.append(player['player_id'])
                players.remove(player['player_id'])
                position_counts[player['position']] += 1
            # Stop if we've selected enough players
            if len(other_managers_picks) == len(teams) - 1: 
                break
    return players, other_managers_picks

chosen_players = []
for round in range(6): # each manager picks 6 players
    # Reset position counts for this round
    position_counts = {'QB': 0, 'WR': 0, 'RB': 0, 'TE': 0}
    for team in teams:
        players = all_players.copy() # start with all players available
        other_managers_picks = []
        if team != 'Manager_1':
            players, other_managers_picks = pick_other_managers(players, players_rank)
        else: # if team == 'Manager_1'
            for pick in other_managers_picks:
                if pick in players:
                    players.remove(pick)

        # Create the 'prob' variable to contain the problem data
        prob = LpProblem("Fantasy_Football_Optimization", LpMaximize)

        player_vars = LpVariable.dicts("Player",players,0,1,LpBinary)
        
        # The objective function is added to 'prob' first
        prob += lpSum([player_vars[i] for i in players if player_data[(player_data['player_id'] == i)]['position_group'].values[0] == pos]) <= position_limits[pos]


        
        # We can enter the constraints that relate to the maximum of each position 
        for pos in position_limits.keys():
            prob += lpSum([player_vars[i] for i in players if player_data[(player_data['player_id'] == i)]['position_group'].item() == pos]) <= position_limits[pos]
        
        # Ensure only 6 players are picked
        prob += lpSum([player_vars[i] for i in players]) == 6

        # The problem is solved using PuLP's choice of Solver
        prob.solve()

        for v in prob.variables():
            if v.varValue == 1:
                chosen_players.append((team, v.name, round+1))
                all_players.remove(v.name)

chosen_players_df = pd.DataFrame(chosen_players, columns=['Manager', 'Player', 'Round'])
print(chosen_players_df)


NameError: name 'pos' is not defined

In [354]:
# Erstellen Sie eine Liste aller möglichen Wochen
all_weeks = list(range(1, 18))

# Erstellen Sie eine Liste aller eindeutigen Spieler-IDs
all_players = player_data['player_id'].unique()

# Erstellen Sie einen neuen DataFrame mit allen Kombinationen von Spielern und Wochen
index = pd.MultiIndex.from_product([all_players, all_weeks], names=['player_id', 'week'])
full_data = pd.DataFrame(index=index).reset_index()

# Führen Sie den neuen DataFrame mit dem ursprünglichen DataFrame zusammen
player_data = pd.merge(full_data, player_data, on=['player_id', 'week'], how='left')

# Füllen Sie die fehlenden Werte
for column in ['player_display_name', 'position_group', 'recent_team', 'season']:
    player_data[column].fillna(method='ffill', inplace=True)

player_data.fillna(0, inplace=True)

In [355]:
nfl_teams = clean_date['home_team'].unique().tolist()
weeks= all_weeks
 # Erstellen Sie ein Dictionary, das jedem Spieler-Team-Kombination die Stärke des Gegners zuweist
opponent_strength = {}
for _, row in clean_date.iterrows():
    # Fügen Sie die Punkte für das Heimteam hinzu
    opponent_strength[(row['week'], row['home_team'])] = row['away_score']
    # Fügen Sie die Punkte für das Auswärtsteam hinzu
    opponent_strength[(row['week'], row['away_team'])] = row['home_score']
# Eine Liste aller einzigartigen Spieler erstellen
players = player_data['player_id'].unique()

# Werte für alle Teams und Wochen festlegen
for team in nfl_teams:
    for week in weeks:
        if (week, team) not in opponent_strength:
            # Wenn das Team in dieser Woche eine Bye Week hat, 
            # setzen Sie die Stärke des Gegners auf einen hohen Wert
            opponent_strength[(week, team)] = 1e10

In [28]:
opponent_strength

{(1, 'CHI'): 10.0,
 (1, 'GB'): 3.0,
 (1, 'CAR'): 30.0,
 (1, 'LA'): 27.0,
 (1, 'CLE'): 43.0,
 (1, 'TEN'): 13.0,
 (1, 'JAX'): 40.0,
 (1, 'KC'): 26.0,
 (1, 'MIA'): 59.0,
 (1, 'BAL'): 10.0,
 (1, 'MIN'): 12.0,
 (1, 'ATL'): 28.0,
 (1, 'NYJ'): 17.0,
 (1, 'BUF'): 16.0,
 (1, 'PHI'): 27.0,
 (1, 'WAS'): 32.0,
 (1, 'LAC'): 24.0,
 (1, 'IND'): 30.0,
 (1, 'SEA'): 20.0,
 (1, 'CIN'): 21.0,
 (1, 'ARI'): 27.0,
 (1, 'DET'): 27.0,
 (1, 'DAL'): 17.0,
 (1, 'NYG'): 35.0,
 (1, 'TB'): 31.0,
 (1, 'SF'): 17.0,
 (1, 'NE'): 3.0,
 (1, 'PIT'): 33.0,
 (1, 'NO'): 28.0,
 (1, 'HOU'): 30.0,
 (1, 'OAK'): 16.0,
 (1, 'DEN'): 24.0,
 (2, 'CAR'): 20.0,
 (2, 'TB'): 14.0,
 (2, 'BAL'): 17.0,
 (2, 'ARI'): 23.0,
 (2, 'CIN'): 41.0,
 (2, 'SF'): 17.0,
 (2, 'DET'): 10.0,
 (2, 'LAC'): 13.0,
 (2, 'GB'): 16.0,
 (2, 'MIN'): 21.0,
 (2, 'HOU'): 12.0,
 (2, 'JAX'): 13.0,
 (2, 'MIA'): 43.0,
 (2, 'NE'): 0.0,
 (2, 'NYG'): 28.0,
 (2, 'BUF'): 14.0,
 (2, 'PIT'): 28.0,
 (2, 'SEA'): 26.0,
 (2, 'TEN'): 19.0,
 (2, 'IND'): 17.0,
 (2, 'WAS'): 31.0,
 (2, 'D

In [356]:
player_data['points_per_strength'] = player_data['predicted_fantasy_points'] / player_data.apply(lambda row: opponent_strength[(row['week'], row['recent_team'])] + 1e-10, axis=1)
predicted_points_per_strength = {(player, week): points_per_strength for player, week, points_per_strength in zip(player_data['player_id'], player_data['week'], player_data['points_per_strength'])}

In [357]:
player_data = player_data.reset_index()

In [38]:
expected_points_long = expected_points.reset_index().melt(id_vars='player_id', var_name='week', value_name='points')
expected_points_long['week'] = expected_points_long['week'].astype(int)  # Stellen Sie sicher, dass 'week' ein integer ist

In [39]:
expected_points_dict = {(row['player_id'], row['week']): row['points'] for index, row in expected_points_long.iterrows()}

In [36]:
from pulp import *
import numpy as np
import pandas as pd

# Liste der Spieler und Teams erstellen
all_players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
position = dict(zip(player_data['player_id'], player_data['position_group']))
players_rank['position'] = players_rank['player_id'].apply(lambda x: position.get(x))

# Update players_rank DataFrame to include only available players
players_rank = players_rank[players_rank['player_id'].isin(all_players)]
position_limits = {'QB': 1, 'WR': 2, 'RB': 2, 'TE': 1}

# To keep track of players picked by each manager in each position
picked_players = {team: {pos: [] for pos in position_limits.keys()} for team in teams}

def pick_other_managers(players, players_rank):
    other_managers_picks = []
    position_counts = {'QB': 0, 'WR': 0, 'RB': 0, 'TE': 0}  # Counts for each position

    for _, player in players_rank.iterrows():
        if player['player_id'] in players:
            # Check if adding this player would violate the position restrictions
            if position_counts[player['position']] < position_limits[player['position']]:
                other_managers_picks.append(player['player_id'])
                players.remove(player['player_id'])
                position_counts[player['position']] += 1
            # Stop if we've selected enough players
            if len(other_managers_picks) == len(teams) - 1: 
                break
    return players, other_managers_picks


# Optimization
chosen_players = []
for round in range(6): # each manager picks 6 players
    # Reset position counts for this round
    position_counts = {'QB': 0, 'WR': 0, 'RB': 0, 'TE': 0}
    for t in teams:
        players = all_players.copy() # start with all players available
        other_managers_picks = []
        if t != 'Manager_1':
            players, other_managers_picks = pick_other_managers(players, players_rank)
        else: # if t == 'Manager_1'
            for pick in other_managers_picks:
                if pick in players:
                    players.remove(pick)

        # optimization for each manager
        # calculate expected points for each manager in each round
        expected_points_others = sum(expected_points_dict[(p, week)] for p in picked_players if p in players)
        # calculate the minimum points needed to win for the round
        min_points_win = expected_points_others + 1

        x = pulp.LpVariable.dicts("chosen", players, cat='Binary')
        prob = LpProblem(f"Fantasy_Football_Optimization_{round}", LpMaximize)
        # change expected points_dict according to the week
        prob += lpSum([expected_points_dict[(player, week)] * x[player] for player in players])  # objective function

        prob += lpSum([x[player] for player in players]) == 1
        prob += lpSum([expected_points_dict[(player, week)] * x[player] for player in players]) >= min_points_win

        for pos, limit in position_limits.items():
            prob += lpSum([x[player] for player in players if position[player] == pos]) <= limit - position_counts[pos]

        prob += lpSum([x[player] for player in players]) <= sum(position_limits.values()) - sum(len(picked_players[t][pos]) for pos in position_limits.keys())

        # solve problem
        status = prob.solve()
        print(LpStatus[status])
        for player in players:
            if x[player].varValue > 0.5:  # this player was chosen
                picked_players[t][position[player]].append(player)
                # remove picked player from player pool
                all_players.remove(player)
        chosen_players.append((t, player, round+1))

chosen_players_df = pd.DataFrame(chosen_players, columns=['Manager', 'Player', 'Round'])
chosen_players_df


NameError: name 'week' is not defined

In [392]:
DMPlayer = np.empty((len(managers), 5), dtype=object)  # 5 Spieler pro Manager
predicted_points = pd.Series(player_data.fantasy_points.values, index=[player_data.player_id, player_data.player_display_name]).to_dict()
# Iteration über jede Runde
for r in range(5):
    # In jeder Runde wechseln sich die Manager ab
    for m in range(len(managers)):
        # Wenn es eine gerade Runde ist, wählt Manager 1 zuerst aus
        # Wenn es eine ungerade Runde ist, wählt Manager 10 zuerst aus
        manager_order = m if r % 2 == 0 else len(managers) - m - 1
        manager_name = 'Manager' + str(manager_order + 1)

        # Problem-Objekt erstellen
        prob = LpProblem("FantasyFootball", LpMaximize)

        # Entscheidungsvariablen definieren
      
        players_vars = LpVariable.dicts("Players", ((i, j) for i in players for j in managers), cat='Binary')


        # Zielfunktion definieren
        prob += lpSum([players_vars[i, j]*predicted_points[(i, week)] for i in players for j in managers for week in weeks if (i, week) in predicted_points])


        # Hinzufügen der aktuellen Runden- und Manager-Einschränkungen
        # Hinzufügen der aktuellen Runden- und Manager-Einschränkungen
        for week in weeks:
            for j in managers:
                prob += lpSum([players_vars[i, j]*predicted_points[(i, week)] for i in players if (i, week) in predicted_points]) >= lpSum([predicted_points[(i,week)] / (opponent_strength[(week, player_teams[i])] + 1e-10) for i in players if (i, week) in predicted_points])

        # Positionseinschränkungen für jedes Team definieren
        for manager in managers:
            prob += lpSum([players_vars[i, manager] for i in players if position[i] == 'QB']) == 1
            prob += lpSum([players_vars[i, manager] for i in players if position[i] == 'WR']) == 2
            prob += lpSum([players_vars[i, manager] for i in players if position[i] == 'RB']) == 2
            prob += lpSum([players_vars[i, manager] for i in players if position[i] == 'TE']) == 1


        # Einschränkung definieren, dass ein Spieler nur einmal gedraftet werden kann
        # Ensure that each player is only picked once.
        for i in players:
            prob += lpSum([players_vars[i,j] for j in managers]) <= 1




        # Problem lösen
        prob.solve()

        # Finden Sie den neu ausgewählten Spieler
        new_player = max([v for v in prob.variables() if "Players" in v.name and value(v) > 0], key=lambda v: value(v))

        # Der Spielername ist der Teil der Variablennamen nach "Players_"
        player_name_raw = new_player.name.split("_", 1)[1]

        # Entfernen Sie die unerwünschten Zeichen
        player_name = player_name_raw.replace("'", "")

        DMPlayer[manager_order][r] = player_name

        # Aktualisieren Sie das Optimierungsproblem, um den neu ausgewählten Spieler aus der verfügbaren Spielerliste zu entfernen
        players = list(player_data['player_id'].unique())


# Ausgabe der Ergebnisse

print("Optimale Auswahl der Spieler:")
for v in prob.variables():
    if value(v) > 0 and 'Players' in v.name:
        print(v.name, "=", value(v))
print("Zielfunktion:", value(prob.objective)) 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mariu/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/7fb838aeb2474a28bc99a491b87e426e-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/7fb838aeb2474a28bc99a491b87e426e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 697 COLUMNS
At line 19979 RHS
At line 20672 BOUNDS
At line 25494 ENDATA
Problem MODEL has 692 rows, 4821 columns and 9640 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.02 seconds
Cgl0004I processed model has 522 rows, 4820 columns (4820 integer (4820 of which binary)) and 9640 elements
Cbc0045I No integer variables out of 4820 objects (4820 integer) have costs
Cbc0045I branch on satisfied N create fake objective Y random cost Y
C

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [272]:
def draft_round(players, available_players, round_number, pick_order):
    """
    Simulates a round of drafting.

    :param players: List of player_ids for the team that we're simulating.
    :param available_players: A list of available players.
    :param round_number: Current round number.
    :param pick_order: List that determines the pick order for each team.
    :returns: The players on our team after the round.
    """
    teams = list(range(1, 11))  # There are 10 teams

    # Loop over each pick in the round
    for i in range(len(teams)):
        team = pick_order[i]
        # If it's our team's turn, we pick the best available player
        if team == my_team:
            players, available_players = pick_best_player(players, available_players, round_number)
        # Otherwise, another manager makes a pick
        else:
            available_players, picked_player = pick_other_managers(available_players)
            data.append({'Manager': 'Manager_' + str(team), 'Player': picked_player, 'Round': round_number})

    return players, available_players


def pick_best_player(players, available_players, round_number):
    """
    Adds the best player to our team and remove it from the available players.

    :param players: List of player_ids for our team.
    :param available_players: A list of available players.
    :param round_number: Current round number.
    :returns: The players on our team and available players after the pick.
    """
    best_player = available_players[0]
    players.append(best_player)
    available_players.remove(best_player)
    data.append({'Manager': 'Manager_' + str(my_team), 'Player': best_player, 'Round': round_number})

    return players, available_players


def pick_other_managers(available_players):
    """
    Simulates the pick of the other managers.

    :param available_players: A list of available players.
    :returns: The available players after the pick and picked player.
    """
    picked_player = available_players[0]
    available_players.remove(picked_player)

    return available_players, picked_player


def run_draft(players, available_players, pick_orders):
    """
    Simulates a draft.

    :param players: List of player_ids for our team.
    :param available_players: A list of available players.
    :param pick_orders: A list of lists, where each list is the pick order for a round.
    :returns: The players on our team after the draft.
    """
    for i in range(len(pick_orders)):
        players, available_players = draft_round(players, available_players, i+1, pick_orders[i])

    return players


In [273]:
# List of all available players

# List for storing the picks of all managers
data = []

# Order in which managers pick players
pick_orders = [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
               [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
               [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
               [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
               [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
               [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]]

# Your team
my_team = 1

# Initialize list to hold your players
my_players = []

# Run the draft
my_players = run_draft(my_players, all_players, pick_orders)

# Display the picks of all managers
df = pd.DataFrame(data)
print(df)


       Manager      Player  Round
0    Manager_1  00-0021206      1
1    Manager_2  00-0022127      1
2    Manager_3  00-0022787      1
3    Manager_4  00-0022803      1
4    Manager_5  00-0022921      1
5    Manager_6  00-0022924      1
6    Manager_7  00-0022943      1
7    Manager_8  00-0023500      1
8    Manager_9  00-0023564      1
9   Manager_10  00-0024221      1
10  Manager_10  00-0024243      2
11   Manager_9  00-0024389      2
12   Manager_8  00-0025396      2
13   Manager_7  00-0025399      2
14   Manager_6  00-0025418      2
15   Manager_5  00-0025708      2
16   Manager_4  00-0026035      2
17   Manager_3  00-0026158      2
18   Manager_2  00-0026189      2
19   Manager_1  00-0026293      2
20   Manager_1  00-0026498      3
21   Manager_2  00-0026544      3
22   Manager_3  00-0026625      3
23   Manager_4  00-0026986      3
24   Manager_5  00-0027029      3
25   Manager_6  00-0027061      3
26   Manager_7  00-0027150      3
27   Manager_8  00-0027685      3
28   Manager_9

In [297]:
import random
from pulp import LpProblem, LpVariable, LpInteger, lpSum, LpMaximize, LpStatus, value

def get_pick_orders(round, managers):
    # Erstellen Sie eine Liste mit allen Managern
    manager_list = list(managers)

    # In ungeraden Runden wird die Reihenfolge zufällig gewählt
    if round % 2 == 1:
        random.shuffle(manager_list)

    # In geraden Runden wird die Reihenfolge umgekehrt
    else:
        manager_list.reverse()

    return manager_list


def get_win_probability(player):
    # Extrahieren Sie die erwarteten Fantasiepunkte aus den Daten des Spielers
    expected_fantasy_points = expected_points_dict[player][1]
    return expected_fantasy_points

def get_player_position(player):
    # Extrahieren Sie die Position des Spielers aus den Spielerdaten
    position = player_data.loc[player_data['player_id'] == player, 'position_group'].values[0]
    return position

def get_available_players():
    # Extrahieren Sie die Liste der verfügbaren Spieler aus den Spielerdaten
    available_players = list(player_data['player_id'])
    return available_players

def optimize_picks(my_players, available_players, pick_orders):
    prob = LpProblem("FantasyFootball", LpMaximize)

    # Variablen erstellen
    player_round_vars = LpVariable.dicts("pick", (available_players, range(len(pick_orders))), 0, 1, LpInteger)

    # Zielfunktion: Maximieren Sie die erwarteten Fantasiepunkte
    prob += lpSum([get_win_probability(p) * player_round_vars[p][r] for p in available_players for r in range(len(pick_orders))])

    # Eine Einschränkung hinzufügen: Jeder Spieler kann nur einmal ausgewählt werden
    for p in available_players:
        prob += lpSum([player_round_vars[p][r] for r in range(len(pick_orders))]) <= 1

    # Eine Einschränkung hinzufügen: In jeder Runde wird nur ein Spieler ausgewählt
    for r in range(len(pick_orders)):
        prob += lpSum([player_round_vars[p][r] for p in available_players]) == 1

    # Eine Einschränkung hinzufügen: Die Positionsanforderungen müssen erfüllt sein
    prob += lpSum([player_round_vars[p][r] for p in available_players if get_player_position(p) == 'QB']) == 1
    prob += lpSum([player_round_vars[p][r] for p in available_players if get_player_position(p) == 'WR']) == 2
    prob += lpSum([player_round_vars[p][r] for p in available_players if get_player_position(p) == 'RB']) == 2
    prob += lpSum([player_round_vars[p][r] for p in available_players if get_player_position(p) == 'TE']) == 1

    # Problem lösen
    prob.solve()

    # Aktualisieren Sie die Liste der ausgewählten Spieler
    for p in available_players:
        for r in range(len(pick_orders)):
            if value(player_round_vars[p][r]) > 0:
                my_players.append(p)

    return my_players

# Führen Sie die Optimierung für jede Runde der Auswahl durch
managers = ["Manager_1", "Manager_2", "Manager_3", "Manager_4", "Manager_5", 
            "Manager_6", "Manager_7", "Manager_8", "Manager_9", "Manager_10"]
number_of_rounds = 6
my_players = []
for round in range(number_of_rounds):
    print(f"Starting optimization for round {round+1} of {number_of_rounds}...")
    # Aktualisieren Sie die Liste der verfügbaren Spieler und die Pick-Reihenfolge
    available_players = get_available_players()
    pick_orders = get_pick_orders(round, managers)

    # Führen Sie die Optimierung durch
    my_players = optimize_picks(my_players, available_players, pick_orders)
    print(f"Completed optimization for round {round+1}. My players now are: {my_players}")


Starting optimization for round 1 of 6...


KeyError: '00-0019596'

In [305]:
# Spieler-Datenstruktur für 'expected_points' anpassen
expected_points = {(row[0], week): points for ((row[0], week), points) in expected_points_dict.items()}

# Spieler-Datenstruktur für die 'y' Variablen anpassen
y = pulp.LpVariable.dicts("y", (managers, [player[0] for player in players]), 0, 1, pulp.LpBinary)


In [329]:
import pulp
# ...vorheriger Code...

# Erstelle players_rank DataFrame
players_rank = pd.DataFrame({"player_id": player_ids, "display_player_name": display_player_names, "position_group": position_groups, "predicted_points": predicted_points})
players_rank = players_rank.set_index('player_id')



# Datenverarbeitung
expected_points = expected_points_dict
players_ids = list(set([player_id for player_id, week in expected_points.keys()])) # get unique player ids
managers = manager_schedule['Manager'].unique().tolist()

# Check if 'Manager_1' and '00-0031589' are in the lists
assert 'Manager_1' in managers, "Manager_1 not in managers list"
assert '00-0031589' in players_ids, "00-0031589 not in players_ids list"

# Set 'player_id' as index for 'players_rank'
players_rank.set_index('player_id', inplace=True)

# Rest of your code...


# Optimierungsproblem definieren
prob = pulp.LpProblem("FantasyFootball", pulp.LpMaximize)

# Entscheidungsvariablen
y = pulp.LpVariable.dicts("y", (managers, players_ids), 0, 1, pulp.LpBinary)

# Zielfunktion
prob += pulp.lpSum([y['Manager_1'][player_id] * expected_points.get((player_id, 1), 0) for player_id in players_ids])

# Beschränkungen
# Spielerbeschränkung pro Manager
for manager in managers:
    prob += pulp.lpSum([y[manager][player_id] for player_id in players_ids]) <= 6

# Positionsbeschränkung
positions = ["QB", "RB", "WR", "TE"]
# Positionsbeschränkungen
position_limit = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'DST': 1, 'K': 1}

for manager in managers:
    for pos in positions:
        # Spieler, die der aktuellen Position entsprechen
        players_at_pos = players_rank[players_rank['position_group'] == pos]['player_id']
        prob += pulp.lpSum([y[manager][player_id] for player_id in players_at_pos]) <= position_limit[pos]

# Optimierungsproblem lösen
prob.solve()

  

# Nur ein Manager kann einen bestimmten Spieler draften
for player_id in players_ids:
    prob += pulp.lpSum([y[manager][player_id] for manager in managers]) <= 1

# 1. Optimierung für Manager_1
prob.solve()

# Drucke die Picks für Manager_1
print("Manager_1's Team:")
for player_id in players_ids:
    if pulp.value(y['Manager_1'][player_id]) == 1:
        print(players_rank.loc[player_id, 'player_display_name'], "(" + players_rank.loc[player_id, 'position_group'] + ")")

# Erstelle eine Liste der von Manager_1 ausgewählten Spieler
picked_players = [player_id for player_id in players_ids if pulp.value(y['Manager_1'][player_id]) == 1]

# Entferne die von Manager_1 ausgewählten Spieler aus der Liste
players_rank = players_rank.loc[~players_rank.index.isin(picked_players), :]

# Aktualisiere die players_ids Liste auf die Spieler, die noch nicht ausgewählt wurden
players_ids = players_rank.index.tolist()

# 2. Simulation für die anderen Manager
managers.remove('Manager_1')  # Entferne Manager_1 aus der Liste

# Definiere die Picks für jeden Manager
picks = {manager: [] for manager in managers}

# Definiere die Positionslimits für jeden Manager
position_counts = {manager: {position: 0 for position in positions} for manager in managers}

# Gehe durch jeden Spieler, beginnend mit dem höchstbewerteten
for _, player_row in players_rank.sort_values(by="rank").iterrows():
    player_id = player_row['player_id']
    player_position = player_row['position_group']

    # Gehe durch jeden Manager
    for manager in managers:
        # Überspringe den Manager, wenn er die Positionslimit erreicht hat
        if position_counts[manager][player_position] >= position_limit[player_position]:
            continue

        # Überspringe den Manager, wenn er bereits genug Spieler ausgewählt hat
        if len(picks[manager]) >= player_limit:
            continue

        # Füge den Spieler zum Team des Managers hinzu und aktualisiere die Positionszähler
        picks[manager].append(player_id)
        position_counts[manager][player_position] += 1

        # Da der Spieler ausgewählt wurde, gehe zum nächsten Spieler
        break

# Drucke die Picks für die anderen Manager
for manager in managers:
    print(manager + "'s Team:")
    for player_id in picks[manager]:
        print(players_rank.loc[player_id, 'player_display_name'], "(" + players_rank.loc[player_id, 'position_group'] + ")")





NameError: name 'player_ids' is not defined

In [321]:
players_rank

,player_id,player_display_name,position_group,predicted_fantasy_points,opportunity_cost,final_score,rank,position
398,00-0033280,Christian McCaffrey,RB,23.891197,5.907076,18.495960,1.0,RB
646,00-0034796,Lamar Jackson,QB,22.539255,3.464991,16.816975,2.0,QB
372,00-0033077,Dak Prescott,QB,19.074265,1.513687,13.806091,3.0,QB
334,00-0032764,Derrick Henry,RB,17.984121,1.203814,12.950028,4.0,RB
722,00-0035228,Kyler Murray,QB,17.560577,0.323530,12.389463,5.0,QB
...,...,...,...,...,...,...,...,...
434,00-0033471,Isaiah McKenzie,WR,2.606819,0.000000,1.824774,793.0,WR
751,00-0035311,Tommy Sweeney,TE,2.606819,0.000000,1.824774,793.0,RB
286,00-0032227,Tyler Kroft,TE,2.606819,0.000000,1.824774,793.0,TE
732,00-0035251,Devin Singletary,RB,2.606819,0.000000,1.824774,793.0,QB


In [306]:
import pandas as pd
import pulp

# Datenverarbeitung
expected_points = {(k[0], k[1]): v for k, v in expected_points_dict.items()}
players = list(expected_points.keys())
managers = manager_schedule['Manager'].unique().tolist()

# Spielerpositionen und Ranking extrahieren
player_positions = player_data.set_index('player_id')['position_group'].to_dict()
players_ranks = players_rank.set_index('player_id')['rank'].to_dict()
ranked_players = players_rank['player_id'].tolist()
# Maximale Anzahl der Spieler und Positionsbeschränkungen
max_players = 6
position_limits = {"QB": 1, "RB": 2, "WR": 2, "TE": 1}



# MILP-Problem erstellen
prob = pulp.LpProblem("FantasyFootball", pulp.LpMaximize)

# Variablen
y = pulp.LpVariable.dicts("y", (managers, players), 0, 1, pulp.LpBinary)

# Zielfunktion
prob += pulp.lpSum([y['Manager_1', player] * expected_points[player] for player in players])

# Beschränkungen
# Spielerbeschränkung pro Manager
for manager in managers:
    prob += pulp.lpSum([y[manager, player] for player in players]) <= max_players

# Positionsbeschränkung pro Manager
for manager in managers:
    for position, limit in position_limits.items():
        prob += pulp.lpSum([y[manager, player] for player in players if player_positions[player[0]] == position]) <= limit

# Beschränkung: Ein Spieler kann nur einmal ausgewählt werden
for player in players:
    prob += pulp.lpSum([y[manager, player] for manager in managers]) <= 1

# Manager picken nach der Rangliste
for i, player in enumerate(ranked_players):
    for manager in managers:
        prob += y[manager, (player, 1)] <= pulp.lpSum([1 - y[m, (ranked_players[j], 1)] for m in managers for j in range(i)])

# Problem lösen
prob.solve()

# Ergebnisse
picked_players = pd.DataFrame([(m, p[0], pulp.value(y[m, p])) for m in managers for p in players if pulp.value(y[m, p]) == 1], columns=['Manager', 'Player', 'Picked'])
print(picked_players)


KeyError: ('Manager_1', ('00-0019596', 1))

In [265]:
from pulp import *
import numpy as np
import pandas as pd



# Liste der Spieler und Teams erstellen
all_players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
position = dict(zip(player_data['player_id'], player_data['position_group']))
players_rank['position'] = players_rank['player_id'].apply(lambda x: position.get(x))

# Update players_rank DataFrame to include only available players
players_rank = players_rank[players_rank['player_id'].isin(all_players)]
position_limits = {'QB': 1, 'WR': 2, 'RB': 2, 'TE': 1}

# To keep track of players picked by each manager in each position
picked_players = {team: {pos: [] for pos in position_limits.keys()} for team in teams}

def pick_other_managers(players, players_rank):
    other_managers_picks = []
    position_counts = {'QB': 0, 'WR': 0, 'RB': 0, 'TE': 0}  # Counts for each position

    for _, player in players_rank.iterrows():
        if player['player_id'] in players:
            # Check if adding this player would violate the position restrictions
            if position_counts[player['position']] < position_limits[player['position']]:
                other_managers_picks.append(player['player_id'])
                players.remove(player['player_id'])
                position_counts[player['position']] += 1
            # Stop if we've selected enough players
            if len(other_managers_picks) == len(teams) - 1: 
                break
    return players, other_managers_picks

# Optimization
chosen_players = []
for round in range(6): # each manager picks 6 players
    for t in teams:
        players = all_players.copy() # start with all players available
        if t != 'Manager_1':
            players, other_managers_picks = pick_other_managers(players, players_rank)
        else: # if t == 'Manager_1'
            for pick in other_managers_picks:
                if pick in players:
                    players.remove(pick)

        # optimization for each manager
        # calculate expected points for each manager in each round
        expected_points_others = sum(expected_points_dict[(p, week)] for p in picked_players if p in players)
        # calculate the minimum points needed to win for the round
        min_points_win = expected_points_others + 1

        x = pulp.LpVariable.dicts("chosen", players, cat='Binary')
        prob = LpProblem(f"Fantasy_Football_Optimization_{round}", LpMaximize)
        # change objective function to maximize the difference between your points and the minimum points to win
        prob += lpSum([x[i] * (expected_points_dict[(i, week)] - min_points_win) for i in players])

        # add constraints
        for pos in position_limits.keys():
            # Subtract the number of players already picked by this manager in this position
            prob += lpSum([x[i] for i in players if position[i] == pos]) == position_limits[pos] - len(picked_players[t][pos])
        prob += lpSum([x[i] for i in players]) == sum(position_limits.values()) - sum(len(picked_players[t][pos]) for pos in position_limits.keys())

        # solve problem
        status = prob.solve()
        print(LpStatus[status])
        for player in players:
            if x[player].varValue > 0.5:  # this player was chosen
                picked_players[t][position[player]].append(player)
                # remove picked player from player pool
                all_players.remove(player)
  # all_players += picked_players[t][position[player]]  # Add picked players back into pool for next manager
        chosen_players.append((t, player, round+1))

chosen_players_df = pd.DataFrame(chosen_players, columns=['Manager', 'Player', 'Round'])
chosen_players_df

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mariu/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/a7169474c5034d06ab23549d260e99e8-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/a7169474c5034d06ab23549d260e99e8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 2716 RHS
At line 2722 BOUNDS
At line 3264 ENDATA
Problem MODEL has 5 rows, 541 columns and 1082 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 107.605 - 0.00 seconds
Cgl0004I processed model has 3 rows, 339 columns (339 integer (332 of which binary)) and 678 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -107.605
Cbc0038I Cleaned solution of -107.605
Cbc0038I Before mini br

,Manager,Player,Round
0,Manager_10,00-0035624,1
1,Manager_2,00-0035624,1
2,Manager_4,00-0035624,1
3,Manager_6,00-0035624,1
4,Manager_8,00-0035624,1
5,Manager_1,00-0035624,1
6,Manager_3,00-0035624,1
7,Manager_5,00-0035624,1
8,Manager_7,00-0035624,1
9,Manager_9,00-0035624,1


In [219]:
manager_schedule

,week,Manager,Opponent
0,1,1,3
1,1,2,4
2,1,3,3
3,1,4,7
4,1,5,6
...,...,...,...
125,13,6,3
126,13,7,5
127,13,8,5
128,13,9,4


In [70]:
for manager, positions in picked_players.items():
    print(f"Team von {manager}:")
    for position, players in positions.items():
        print(f"  {position}:")
        for player in players:
            print(f"    {player}")
    print()


NameError: name 'picked_players' is not defined

In [333]:
for manager in picked_players.keys():
    print(f"Team von {manager}:")
    manager_df = chosen_players_df[chosen_players_df['Manager'] == manager]
    for _, row in manager_df.iterrows():
        print(f"  Spieler {row['Player']} wurde in Runde {row['Round']} ausgewählt.")
    print()


Team von Manager_10:
  Spieler 00-0035624 wurde in Runde 1 ausgewählt.
  Spieler 00-0035624 wurde in Runde 2 ausgewählt.
  Spieler 00-0035624 wurde in Runde 3 ausgewählt.
  Spieler 00-0035624 wurde in Runde 4 ausgewählt.
  Spieler 00-0035624 wurde in Runde 5 ausgewählt.
  Spieler 00-0035624 wurde in Runde 6 ausgewählt.

Team von Manager_2:
  Spieler 00-0035624 wurde in Runde 1 ausgewählt.
  Spieler 00-0035624 wurde in Runde 2 ausgewählt.
  Spieler 00-0035624 wurde in Runde 3 ausgewählt.
  Spieler 00-0035624 wurde in Runde 4 ausgewählt.
  Spieler 00-0035624 wurde in Runde 5 ausgewählt.
  Spieler 00-0035624 wurde in Runde 6 ausgewählt.

Team von Manager_4:
  Spieler 00-0035624 wurde in Runde 1 ausgewählt.
  Spieler 00-0035624 wurde in Runde 2 ausgewählt.
  Spieler 00-0035624 wurde in Runde 3 ausgewählt.
  Spieler 00-0035624 wurde in Runde 4 ausgewählt.
  Spieler 00-0035624 wurde in Runde 5 ausgewählt.
  Spieler 00-0035624 wurde in Runde 6 ausgewählt.

Team von Manager_6:
  Spieler 00-003

In [199]:
# Calculate scores for each week
scores = []
for week in range(1, 14):
    for t in teams:
        # Get the players this manager has chosen up to this round
        t_players = chosen_players_df[(chosen_players_df['Manager'] == t)]['Player']
        # Calculate the expected points for this manager in this week
        t_points = sum(expected_points_dict[(p, week)] for p in t_players)
        scores.append((t, week, t_points))

scores_df = pd.DataFrame(scores, columns=['Manager', 'Week', 'Expected Points'])

# Now, to find out who wins each match, we would compare the scores for each pair of teams in each week
matches = []
for week in range(1, 14):
    # Iterate over pairs of teams (assumes teams are in pairs for matches)
    for t1, t2 in zip(teams[::2], teams[1::2]):
        t1_points = scores_df[(scores_df['Manager'] == t1) & (scores_df['Week'] == week)]['Expected Points'].item()
        t2_points = scores_df[(scores_df['Manager'] == t2) & (scores_df['Week'] == week)]['Expected Points'].item()
        if t1_points > t2_points:
            matches.append((week, t1, t2, 'win'))
        else:
            matches.append((week, t1, t2, 'loss'))

matches_df = pd.DataFrame(matches, columns=['Week', 'Manager', 'Opponent', 'Result'])
matches_df
scores_df

,Manager,Week,Expected Points
0,10,1,70.600953
1,9,1,67.971461
2,8,1,64.033931
3,7,1,52.460959
4,6,1,60.669792
...,...,...,...
125,5,13,65.962032
126,4,13,59.839766
127,3,13,62.563981
128,2,13,73.347301


In [183]:
# Merge the chosen players with the player_data to get the names and positions
merged_df = pd.merge(chosen_players_df, player_data, left_on='Player', right_on='player_id')

# Filter to get the players chosen by Manager_1
manager_1_players = merged_df[merged_df['Manager'] == '1']

# Display the player names and positions
manager_1_players = manager_1_players[['player_display_name', 'position_group']].drop_duplicates()
manager_1_players

,player_display_name,position_group


In [149]:
import pandas as pd
import numpy as np

# Anzahl der Manager und Wochen
num_managers = 10
num_weeks = 13

# Erzeugen Sie eine zufällige Permutation der Manager für jede Woche
np.random.seed(0)  # Für reproduzierbare Ergebnisse
schedule = np.zeros((num_managers, num_weeks), dtype=int)
for week in range(num_weeks):
    schedule[:, week] = np.random.permutation(num_managers)

# Erzeugen Sie die `manager_schedule` Tabelle
manager_schedule = pd.DataFrame({
    'week': np.repeat(np.arange(num_weeks) + 1, num_managers),
    'Manager': np.tile(np.arange(num_managers) + 1, num_weeks),
    'Opponent': schedule.ravel() + 1,  # Addiere 1, um Manager von 1 bis 10 anstatt von 0 bis 9 zu haben
})

# Stellen Sie sicher, dass jeder Manager genau 13 Spiele spielt
assert all(manager_schedule['Manager'].value_counts() == num_weeks)

print(manager_schedule)


     week  Manager  Opponent
0       1        1         3
1       1        2         4
2       1        3         3
3       1        4         7
4       1        5         6
..    ...      ...       ...
125    13        6         3
126    13        7         5
127    13        8         5
128    13        9         4
129    13       10         1

[130 rows x 3 columns]


In [204]:
from pulp import *
import numpy as np
import pandas as pd

# players, expected_points, schedule and position as before
# Liste der Spieler und Teams erstellen
all_players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
position = dict(zip(player_data['player_id'], player_data['position_group']))
players_rank['position'] = players_rank['player_id'].apply(lambda x: position.get(x))


# Update players_rank DataFrame to include only available players
players_rank = players_rank[players_rank['player_id'].isin(all_players)]
position_limits = {'QB': 1, 'WR': 2, 'RB': 2, 'TE': 1}

# To keep track of players picked by each manager in each position
picked_players = {team: {pos: [] for pos in position_limits.keys()} for team in teams}

def pick_other_managers(players, players_rank):
    other_managers_picks = []
    position_counts = {'QB': 0, 'WR': 0, 'RB': 0, 'TE': 0}  # Counts for each position

    for _, player in players_rank.iterrows():
        if player['player_id'] in players:
            # Check if adding this player would violate the position restrictions
            if position_counts[player['position']] < position_limits[player['position']]:
                other_managers_picks.append(player['player_id'])
                players.remove(player['player_id'])
                position_counts[player['position']] += 1
            # Stop if we've selected enough players
            if len(other_managers_picks) == len(teams) - 1: 
                break
    return players, other_managers_picks

# optimization
# optimization
chosen_players = []

for week in range(1, 14):  # assuming there are 6 weeks of games
    for round in range(6): # each manager picks 6 players
        if round % 2 != 0:  # For every other round, reverse the order of teams (Snake draft)
            teams = teams[::-1]
        for t in teams:
            players = all_players.copy() # start with all players available
            if t != 'Manager_1':
                players, _ = pick_other_managers(players, players_rank)

            # optimization for each manager
            x = pulp.LpVariable.dicts("chosen", players, cat='Binary')
            prob = LpProblem(f"Fantasy_Football_Optimization_{round}", LpMaximize)
            prob += lpSum([x[i] * expected_points_dict[(i, week)] for i in players])
            
            # add constraint: each manager can pick only one player in each round
            prob += lpSum([x[i] for i in players]) == 1

            # solve problem
            prob.solve()
            for player in players:
                if x[player].varValue > 0.5:  # this player was chosen
                    chosen_players.append((t, player, round+1, week))
                    break
            if player in all_players:
                all_players.remove(player)

chosen_players_df = pd.DataFrame(chosen_players, columns=['Manager', 'Player', 'Round', 'Week'])
chosen_players_df


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mariu/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/aa887a9630aa4993a87eab7301b8aafc-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y8/j3l3spt14jd5q6r19v3l8_800000gn/T/aa887a9630aa4993a87eab7301b8aafc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 1920 RHS
At line 1922 BOUNDS
At line 2464 ENDATA
Problem MODEL has 1 rows, 541 columns and 541 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 26.8891 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -26.8891 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of wh

,Manager,Player,Round,Week
0,1,00-0033077,1,1
1,2,00-0033951,1,1
2,3,00-0026498,1,1
3,4,00-0022942,1,1
4,5,00-0028986,1,1
...,...,...,...,...
536,4,00-0027793,6,9
537,3,00-0027688,6,9
538,2,00-0025399,6,9
539,1,00-0021206,6,9


In [202]:
all_managers_df = pd.DataFrame(teams, columns=['Manager'])
wins_df = all_managers_df.merge(wins_df, on='Manager', how='left')
wins_df['Wins'] = wins_df['Wins'].fillna(0).astype(int)  # Replace NaNs with 0
wins_df


,Manager,Wins
0,10,7
1,9,0
2,8,8
3,7,0
4,6,5
5,5,0
6,4,6
7,3,0
8,2,6
9,1,0


In [200]:
# Count wins for each manager
wins = matches_df[matches_df['Result'] == 'win']['Manager'].value_counts()

wins_df = pd.DataFrame(wins).reset_index()
wins_df.columns = ['Manager', 'Wins']
wins_df


,Manager,Wins
0,8,8
1,10,7
2,4,6
3,2,6
4,6,5


In [201]:
# Determine the number of weeks
n_weeks = 13

# Calculate the total points for each manager per week
weekly_points = {manager: [0]*n_weeks for manager in teams}  # assuming n_weeks is the number of weeks
for manager, player, _ in chosen_players:  # we don't need to use the round anymore
    for week in range(n_weeks):
        if (player, week+1) in expected_points_dict:  # get points for player in each week
            weekly_points[manager][week] += expected_points_dict[(player, week+1)]

# Matches dataframe
matches = []

# Go through each week and calculate the scores for each match
for week in range(n_weeks):
    for _, row in manager_schedule[manager_schedule['week'] == week+1].iterrows():
        manager_1, manager_2 = row['Manager'], row['Opponent']
        score_1, score_2 = weekly_points[manager_1][week], weekly_points[manager_2][week]
        
        if score_1 > score_2:
            winner = manager_1
        elif score_2 > score_1:
            winner = manager_2
        else:
            winner = 'Draw'

        matches.append((week+1, manager_1, manager_2, score_1, score_2, winner))

# Create a DataFrame to store the match results
matches_df = pd.DataFrame(matches, columns=['Week', 'Manager 1', 'Manager 2', 'Score 1', 'Score 2', 'Winner'])

matches_df


,Week,Manager 1,Manager 2,Score 1,Score 2,Winner
0,1,1,3,69.792692,60.740396,1
1,1,2,4,76.383141,82.344381,4
2,1,3,3,60.740396,60.740396,Draw
3,1,4,7,82.344381,52.460959,4
4,1,5,6,84.846166,60.669792,5
...,...,...,...,...,...,...
125,13,6,3,57.078806,62.563981,3
126,13,7,5,74.190414,65.962032,7
127,13,8,5,59.615091,65.962032,5
128,13,9,4,59.048933,59.839766,4


In [139]:
# Count the number of wins for each manager
wins = matches_df[matches_df['Winner'] != 'Draw'].groupby('Winner').size()

wins


Winner
Manager_1     10
Manager_10     1
Manager_2     12
Manager_3     10
Manager_4      4
Manager_5     10
Manager_6      5
Manager_7      3
Manager_8      3
Manager_9      3
dtype: int64